In [8]:
#used this as my primary refrence
#https://github.com/kaushaltrivedi/fast-bert
#Fernando Mejia
from pathlib import Path
import torch
import tensorflow as tf

from fast_bert.data_cls import BertDataBunch
from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy
import logging
from fast_bert.prediction import BertClassificationPredictor


In [2]:
tf.test.gpu_device_name()


''

In [3]:
DATA_PATH = Path('data/')
LABEL_PATH = Path('label/')
AUG_DATA_PATH = Path('../data/data_augmentation/')


databunch = BertDataBunch(DATA_PATH, LABEL_PATH,
                          tokenizer='bert-base-uncased',
                          train_file='train.csv',
                          val_file='val.csv',
                          label_file='labels.csv',
                          text_col='text',
                          label_col='author',
                          batch_size_per_gpu=16,
                          max_seq_length=1024,
                          multi_gpu=True,
                          multi_label=False,
                          model_type='bert')

In [4]:
torch.cuda.is_available()


False

In [5]:
logger = logging.getLogger()
device = torch.device("cuda")
metrics = [{'name': 'accuracy', 'function': accuracy}]
OUTPUT_DIR=Path('output/')

learner = BertLearner.from_pretrained_model(
                        databunch,
                        pretrained_path='bert-base-uncased',
                        metrics=metrics,
                        device=device_cuda,
                        logger=logger,
                        output_dir=OUTPUT_DIR,
                        finetuned_wgts_path=None,
                        warmup_steps=500,
                        multi_gpu=True,
                        is_fp16=True,
                        multi_label=False,
                        logging_steps=50)

NameError: name 'device_cuda' is not defined

In [6]:
learner.fit(epochs=6,
            lr=6e-5,
            validate=True, 	# Evaluate the model after each epoch
            schedule_type="warmup_cosine",
            optimizer_type="lamb")
learner.save_model()
#model saved here used for next model

NameError: name 'learner' is not defined

In [ ]:
MODEL_PATH = OUTPUT_DIR/'model_out'

predictor = BertClassificationPredictor(
                model_path=MODEL_PATH,
                label_path=LABEL_PATH, # location for labels.csv file
                multi_label=False,
                model_type='xlnet',
                do_lower_case=False)

output = predictor.predict_batch(list(pd.read_csv("data/test.csv")['text'].values))

pd.DataFrame(output).to_csv('data/results.csv')
results = pd.read_csv('data/results.csv')
print(results)